# Kontejnerizace na úrovni operačního systému

## Co se v této sekci dozvíte?

* svět před kontejnerizací
* možnosti virtualizace jednotlivých prostředků jež jsou používána aplikacemi
* Docker = praktická kontejnerizace
* Docker - základní použití

## Problémy vývoje a nasazení aplikací v běžném OS

Soudobé operační systémy jsou velmi komplexními platformami, které nabízejí velké množství prostředků pro aplikace, které s nimi pracují. 

Jedním z klíčových úkolů operačních systémů je dosáhnnou stavu, kdy má každá aplikace pro svůj vlastní (virtuální) počítač. Výrazně to usnadňuje návrh aplikace a zajišťuje, že se aplikace navzájem negativně) neovlivňují.

Už na úrovni operačního systému tak dochází k virtualiazací klíčových prostředků jako je například procesor, paměť nebo síťové připojení. Mnohé prostředky však zůstávají sdílené:

* souborový systém se sdíleným i soubory např. (dynamickými) knihovnami 
* jmenný prostor TCP/IP portů
* systémové služby v podobě stále běžících procesů (v unixové terminologii démonů)
* systémové programy, jež jsou využívány pro vývoj a běh aplikací (včetně interpretů a překladačů vyšších programovacích jazyků)
* uživatelské účty/skupiny využívané pro běh neinteraktivních procesů, které slouží k detailnějšímu nastavení práv (tj. nejsou využívány reálnými uživateli)
* šířka připojení do sítě, prostor na disku

Dlouhou dobu nepředstavovalo toto rozdělení žádný větší problém, neboť případné kolizi lze zabránit vhodně zvolenými jedinečnými identifikátory (různá jména souborů/adresářů, spustitelných programů, systémových uživatelů, vyhrazené TCP/IP porty apod.)

Tato jednoduchá ad hoc řešení však přestala stačit v okamžiku, kdy:

1. rychlý vývoj vede k nutnosti využití různých verzí sdílených prostředků (např. knihoven, interpretů)
2. je potřeba provozovat najednou několik instancí stejné aplikace (často v různých verzích)
3. je potřeba aplikaci hostovat v různých verzích/distribucích operačního systému resp. v různých operačních sytémech

Bod (1) je typický pro většinu dnešních systému, aplikací a knihoven, u nichž vývoj běžně vede k nekompatibilitám, změnám rozhraní, apod. (tím spíše, že v mnoha případech neexistují žádné standardy). Potřeba souběžného spouštění více instancí (2) je typická pro cloudové služby (platform as service, PaaS), hostování v různých variantách OS (3) je klíčová především ve světe Linuxu, kde neexistuje centrální autorita pro podstatné části bakalářské práce.

Mezi typické problémy související s prostředky, které nejsou virtualizovány na úrovni operační paměti:

* použití nekompatibilní verze knihovny/interpretu
* kolize v konfiguračních souborech
* kolize využití TCP/IP portů (více síťových serverů využívá stejný port)
* běh aplikace / systémových programů ovlivňuje jiné aplikace
* kolize práv k souborům a jiným prostředkům (nemáte práva, která potřebujete, resp. máte práva, která mohou ohrozit ostatní)
* omezování jiných procesů alokováním vzácných prostředků (paměti, šířky pásma připojení) 

**Úkol**: S jakými problémy souvisejícími se sdílením prostředků jste se ve své praxi setkaly?

## Kontejnerizace

Řešením je kontejnerizace tj. spuštění aplikací v prostředí, které zajišťuje, že všechny výše uvedené prostředky jsou virtualizovány tj. jsou vytvořeny náhradní prostředky, které nabízejí stejnou funkčnost, ale jsou vyhrazené tj. patří jen dané aplikaci.

Poznámka: kontejnerizace se provádí i na dalších softwarových úrovních a známá je i z hardwaru.

Kontejnerizaci aplikací lze provádět i na na aplikační úrovni, tj. je podporována přímo aplikací. Tento přístup však má několik nevýhod: podpora je většinou komplexní a především se její provedení a rozhraní může u jednotlivých aplikací lišit.

> speciálním případem tohoto typu kontejnerizace jsou virtuální prostředí interpretrů, které nabízejí pro každý projekt prostředí s různou konfigurací překladače, nábídkou modulů a knihoven (např. `conda`).

Podpora na úrovni OS na druhou stranu vyžaduje podporu přímo v jádře OS (jinak nemůže být dostatečně transparentní). Pokud tato podpora chybí je nutné virtualizovat celý OS (kontejner běží v dedikované instanci virtualizovaného OS).  To sice situaci řeší včetně podpory interperability mezi kontejnery (virtuální sítě, mountování sdílených adresářů. apod.) je to však značně neefektivní řešení, které si vede k vytváření mnoha kopií (identického) operačního systému a jeho aplikací. To prakticky znemožňuje souběžnou existenci většího počtu kontejnerizovaných aplikací (např. v řádu stovek).
> I tento způsob kontejnerizace je v mnoha případech vhodným řešením: například tehdy pokud je aplikace závislá na velkém množství knihoven a podpůrných procesů (např. kontejnerizace GUI aplikací), resp. kontejnerizace nezávislá na platformě (některé platformy jinou možnost ani nenabízejí)

**Otázka**: Jaké problémy řeší virtuální prostředí nabízené např. nástrojem `conda` (resp. které neřeší)

### Podpora kontejnerizace v OS Linux

Výrazný pokrok v podpoře kontejnerů přinesl operační systém Linux. Unix z něhož Linux vychází některé elementární prostředky nabízel už od svých počátků (např. tzv. `chroot` umožňují omezení souborového systému na některou z jeho větví). Už s pomocí těchto prostředků byly možno vytvořit kontejnerové systémy (LXC).

Současná virtualizace v Linuxu je založena na následující podpoře v jádře OS:
* *linux kernel namespaces* (virtualizace sdílených prostředků, identifikátorů, apod.)
* podpora *union mounting* (umožňuje kombinovat striktní oddělení se sdílením a to i uvnitř adesářů)
* *cgroups* umožní řídit přidělování procesorového času, paměti, šířky pásma připojení k síti

**Úkol**: Podívejte se na popis jmenných prostorů jádra a diskutujte jejich použitelnost při virtualizaci.

Tyto prostředky jsou navrženy univerzálně a existuje tak několik systémů pro kontejnerizaci na úrovni OS. V současnosti je nejpoužívanějším systémem tohoto druhu `Docker`.

### Docker

**Conquer App Complexity** (motto Dockeru)

Docker je systém pro kontejnerizaci a tak je jeho centrálním objektem **kontejner**. Kontejner je de-facto množina procesů, které běží v určitém jasně definovaném prostředí, jenž mimo jiné obsahuje:

* souborový systém (včetně všech konfiguračních souborů, dat a spustitelných aplikací)
* tcp/ip stack s unikátní siťovou adresou
* systémové služby (démony)
* uživatelské účty (včetně roota)

Všechny tyto prostředky jsou **vyhrazené** (= patřící jen dané instanci kontejneru), některé z nich však lze mapovat na prostředky hostitelské instance OS - například adresáře či síťový subsystém.

Kontejner vzniká spuštěním procesu v rámci tzv. **obrazu** (*image*). Obraz je ve své podstatě primárně souborový systém representovaný více vrstvami, kde každá vrstva vzniká jako výsledek nějakého příkazu. Kromě této (statické) části obraz definuje tzv. startovní příkaz (tj. jaký proces bude spuštěn nad výsledným souborovým systémem při instanciaci kontejneru z obrazu) a jaké bude rozhraní kontejneru s okolím.

Infrastruktura Dockeru obsahuje kromě objektů i infrastrukturu pro jejich správu, která ve své minimální podobě obsahuje:

**docker démon** — správce kontejnerů a obrazů, služba hostitelského operačního systému, jež komunikuje s klinty pomocí síťového Docker API

**docker klient** — nástroj pomocí něhož uživatelé komunikují s docker démonem. Klasický docker klient je konzolová aplikace, jíž jsou všechny parametry přidány na příkazovém řádku.

**docker repositář** (registry) — úložiště docker obrazů. Je kritickou částí docker infrastruktury, neboť obrazy mohou vznikat postupným rozšiřováním tj. přidáváním dalších vrstev. 

### Instalace docker démona a klienta

Instalace je detailně popsána na stránkách https://docs.docker.com/get-docker/. V další části předpokládám, že je využit docker pro Linux. 

**Úkol**: Nainstalujte se docker ve svém oblíbeném operačním systému  (což je předpokládám Linux).

## Spuštění kontejneru

Spuštění kontejneru z obrazu je více než snadné, stačí použít volbu `run` příkazu `docker` a uvést jméno obrazu. Obraz je buď již ve správě lokálního démona nebo je stažen z repositáře (standardně je to repozitář https:hub.docker.com.

V některých případech je nutno využít přepínače. Pro spuštění interaktivního kontejneru (tj. kontejneru, jehož aplikace musí komunikovat přes terminál) jsou to volby `-i` (keep STDIN open) a `-t` (allocate a pseudo-TTY).

Následujícím příkazem spustíme v interaktivním režimu obraz s minimální instalací Alpine Linuxu. Po spuštění by se měl objevit rootovský prompt.

`docker run -it alpine`

**Úkol** Prozkoumejte jak izolaci kontejneru na úrovni souborového systému, procesů (`ps`) a síťové konektivity (`ifconfig`). Jak velký je souborový systém? (`du`)

**Poznámka ke jménům obrazů:** Oficiální obrazy mají jednoduchá jména odpovídající označení aplikace. Na opačném pólu jsou označení typu  `user-name/aplikace:verze`. Mnohá jména jsou jen (stručnější) aliasy jmen delších.

**Úkol**: Jaký je rozdíl mezi `attached` a `detached` režimem běhu kontejneru?

### Správa kontejnerů

Běžící kontejnery lze vypsat příkazem `docker ps` (jou tak chápány trochu jako procesy).

In [3]:
!docker ps

CONTAINER ID   IMAGE     COMMAND     CREATED       STATUS       PORTS     NAMES
b07080839d0e   alpine    "/bin/sh"   2 hours ago   Up 2 hours             thirsty_rosalind


Alternativně lze běžící kolekce zobrazit příkazem  `docker container ls`.

In [4]:
!docker container ls

CONTAINER ID   IMAGE     COMMAND     CREATED       STATUS       PORTS     NAMES
b07080839d0e   alpine    "/bin/sh"   2 hours ago   Up 2 hours             thirsty_rosalind


Příkazy ve tvaru `docker typ-objektu příkaz` jsou novější a nabízejí více možností. Např. zobrazení všech obrazů zajišťuje příkaz:

In [6]:
!docker image ls

REPOSITORY        TAG              IMAGE ID       CREATED        SIZE
jfcz/hadoop       v1               9e7752104de8   2 days ago     234MB
<none>            <none>           58ec8fcb9ec5   2 days ago     1.12GB
jfcz/nosql        v1               4b40cb70d446   2 weeks ago    468MB
<none>            <none>           e64b1a2fe759   2 weeks ago    468MB
<none>            <none>           00b0fa14935a   2 weeks ago    468MB
<none>            <none>           4b631aaab533   2 weeks ago    468MB
<none>            <none>           fe6d3887287a   2 weeks ago    263MB
<none>            <none>           6a7d1fccde89   2 weeks ago    243MB
jfcz-nosql        latest           b1a3199ff822   2 weeks ago    72.8MB
jfcz/nosql        <none>           b1a3199ff822   2 weeks ago    72.8MB
python            buster           062c48fd2234   3 weeks ago    892MB
postgres          latest           1ee973e26c65   3 weeks ago    376MB
jf-python         latest           e8ec234f474d   4 weeks ago    62.1MB
nos

vymazání všech obrazů, které nejsou využívány: `docker image prune` (obrazy jsou dost velké a mohou zbytečně zabírat místo ma disku)

výmaz konkrétního obrazu:

In [12]:
!docker image rm hello-world

Untagged: hello-world:latest
Untagged: hello-world@sha256:97a379f4f88575512824f3b352bc03cd75e239179eea0fecc38e597b2209f49a
Deleted: sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412
Deleted: sha256:e07ee1baac5fae6a26f30cabfe54a36d3402f96afda318fe0a96cec4ca393359


Užitečná je také možnost nedobrovolně ukončovat běžící kontejnery (což zahrnuje zabití procesů), jejich další prostředky však zůstávají. Pro úplné odstranění kontejneru je nutné využít příkazu.

In [13]:
!docker container kill thirsty_rosalind
!docker container rm thirsty_rosalind

Error response from daemon: Cannot kill container: thirsty_rosalind: Container b07080839d0e54767a547f3ef1b22b2270818a438160b3e533c608f291ca1c7f is not running


Kontejnery lze identifikovat jejich jménem (implicitně jsou tvořeny dvěma slovy adjektivum_substantivum, jméno lze uvést pří spuštění) nebo identifikátorem (= počátek sha256 otisku).

Všechny pozastavené kontejnery lze odstranit příkazem `docker container prune`.

Poznámka: Docker kontejnery mohou zaujímat podstatnou část disku a tak je vhodné tu a tam provést prořezání.

### Persistentní kontejnery a komunikace s kontejnerem prostřednictví TCP/IP

Soubory vytvářené při běhu uvnitř kontejneru nejsou standardně persistentní, tj. data při dalším spuštění kontejneru je již nenajdete. Stejně tak jsou kontejnery při standardním spuštění důsledně izolovány, tj. komunikace s nimi není možná ani pomocí sdíleného souborového systému ani pomocí síťového rozhraní.

Jako příklad vezměme oficiální obraz pro postgresql. Bezparametrické spuštění obrazu nefunguje. 

In [15]:
!docker run postgres

Error: Database is uninitialized and superuser password is not specified.
       You must specify POSTGRES_PASSWORD to a non-empty value for the
       superuser. For example, "-e POSTGRES_PASSWORD=password" on "docker run".

       You may also use "POSTGRES_HOST_AUTH_METHOD=trust" to allow all
       connections without a password. This is *not* recommended.

       See PostgreSQL documentation about "trust":
       https://www.postgresql.org/docs/current/auth-trust.html


Doplníme tedy parametr se superuživatelským heslem .

In [16]:
!docker run -e POSTGRES_PASSWORD=password postgres

The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locale "en_US.utf8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "english".

Data page checksums are disabled.

fixing permissions on existing directory /var/lib/postgresql/data ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... Etc/UTC
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /var/lib/postgresql/data -l logfile start

initdb: warning: enabling "trust" authentication for local connections
You can change this by editin

Teď už vše jak se zdá funguje. Má to však dva háky (označení `háček` dostatečně nevyjadřuje míru). Z prvé k databázi se nikdo externě nepřipojí A pokud by se mu to nakonec podařilo, databáze zapomene všechny změny po ukončení běhu kontejneru.

In [18]:
!docker ps

CONTAINER ID   IMAGE      COMMAND                  CREATED          STATUS          PORTS      NAMES
a6e63632d4e8   postgres   "docker-entrypoint.s…"   49 seconds ago   Up 48 seconds   5432/tcp   unruffled_kepler


In [20]:
!docker container kill unruffled_kepler
!docker container rm unruffled_kepler

unruffled_kepler
unruffled_kepler


Plnohodnotné využití těchto kontejnerů je potřeba doplnit ještě dva parametry:
    
1) mapování měněné části souborového systému na (měnitelnou) část souborového systému
2) mapování exportovaných TCP/UDP portů

V našem případě to znamená použít spuštění ve tvaru:

`docker run -p 54320:5432 -v /tmp/datadir:/var/lib/postgresql/data -e POSTGRES_PASSWORD=password postgres`

přepínač `-p` mapuje lokální port 5432 v kontejneru (standardní TCP/IP port Postgresql) na port 54320 na hostitelském počítači (ten může být libovolný, pokud nekoliduje s porty používaqnými službami na hostitelském počítači, v mém případě mi běží lokální nedockerizovaný PostgresSQL a tak jsem port musel změnit).


přepínač `-v` mapuje (mountuje) adresář `/var/lib/postgresql/data` na adresář `/tmp/datadir`  na hostitelském OS (nepříliš vhodně zvolený adresář:). Jaký adresář je nutno mapovat se dozvíte v dokumentaci obrazu (zde https://github.com/docker-library/docs/blob/master/postgres/README.md).

![Jupyter](jupyter.png)